## 最適化に入る前に


In [ ]:
# 普通のリスト
list1 = [1, 2, 3, 4, 5]

# 各要素を2乗したリストを作る➀
list2 = []
for i in list1:
    list2.append(i**2)

list2

In [ ]:
# 各要素を2乗したリストを作る➁
list3 = [i**2 for i in list1]
list3

In [ ]:
# 辞書
# キー：値という関係が保持される
price_dict = {
    'apple':100,
    'banana':200,
    'orange':300
}

# キーを指定すると値を参照できる
price_dict['banana']

In [ ]:
# もう一つ作っておく
# 数量を表す辞書
qty_dict = {
    'apple':2,
    'banana':4,
    'orange':3
}

qty_dict['banana']

In [ ]:
# キーだけ、値だけを取り出すことも可能
fruits = list(qty_dict.keys())
fruits

In [ ]:
# 最適化で頻出！積の和をとる
# それぞれの果物について価格×数量を計算し、その合計を計算している
sum( price_dict[fruit] * qty_dict[fruit] for fruit in fruits )

## データの読み込みと加工
今回は、架空のお菓子のカロリーと価格が載ったデータを扱います。  
数理最適化によって、予算内でなるべくたくさんのカロリーを摂取することを目指します！

In [6]:
import pandas as pd
import pulp      # 毎回「pulp.~」と書くのが面倒なのでこうしています

In [2]:
# データの読み込み
df = pd.read_csv('knapsack_data.csv')
df

,name,kcal,price
0,カンガルーのマーチ,336,84
1,コートジボワールチョコ,309,96
2,ガプリコ,194,102
3,バッケス,336,198
4,ハコダテポテト,446,120
5,いもりこ,299,120
6,かっぱかにせん,486,120
7,フェネットグミ,166,100
8,アルツート,344,102
9,ハッピータン,348,98


In [3]:
# カロリーが高いのは？
df.sort_values('kcal', ascending=False)

,name,kcal,price
14,キャベツ次郎,520,110
6,かっぱかにせん,486,120
4,ハコダテポテト,446,120
16,しんがりコーン,409,147
12,あんドーナツ,387,108
9,ハッピータン,348,98
8,アルツート,344,102
3,バッケス,336,198
0,カンガルーのマーチ,336,84
11,たべっ子アニマル,330,98


In [4]:
# 最適化で記述しやすいように、辞書にしておきます
snack = df.set_index('name').to_dict(orient='index')
snack

{'カンガルーのマーチ': {'kcal': 336, 'price': 84},
 'コートジボワールチョコ': {'kcal': 309, 'price': 96},
 'ガプリコ': {'kcal': 194, 'price': 102},
 'バッケス': {'kcal': 336, 'price': 198},
 'ハコダテポテト': {'kcal': 446, 'price': 120},
 'いもりこ': {'kcal': 299, 'price': 120},
 'かっぱかにせん': {'kcal': 486, 'price': 120},
 'フェネットグミ': {'kcal': 166, 'price': 100},
 'アルツート': {'kcal': 344, 'price': 102},
 'ハッピータン': {'kcal': 348, 'price': 98},
 '柿の素': {'kcal': 209, 'price': 280},
 'たべっ子アニマル': {'kcal': 330, 'price': 98},
 'あんドーナツ': {'kcal': 387, 'price': 108},
 'かば焼き様': {'kcal': 10, 'price': 11},
 'キャベツ次郎': {'kcal': 520, 'price': 110},
 'モンゴルヨーグル': {'kcal': 25, 'price': 22},
 'しんがりコーン': {'kcal': 409, 'price': 147}}

In [5]:
# お菓子の名前と属性を指定すれば値を参照できます
print(snack['いもりこ']['kcal'])
print(snack['いもりこ']['price'])

299
120


In [8]:
snack

{'カンガルーのマーチ': {'kcal': 336, 'price': 84},
 'コートジボワールチョコ': {'kcal': 309, 'price': 96},
 'ガプリコ': {'kcal': 194, 'price': 102},
 'バッケス': {'kcal': 336, 'price': 198},
 'ハコダテポテト': {'kcal': 446, 'price': 120},
 'いもりこ': {'kcal': 299, 'price': 120},
 'かっぱかにせん': {'kcal': 486, 'price': 120},
 'フェネットグミ': {'kcal': 166, 'price': 100},
 'アルツート': {'kcal': 344, 'price': 102},
 'ハッピータン': {'kcal': 348, 'price': 98},
 '柿の素': {'kcal': 209, 'price': 280},
 'たべっ子アニマル': {'kcal': 330, 'price': 98},
 'あんドーナツ': {'kcal': 387, 'price': 108},
 'かば焼き様': {'kcal': 10, 'price': 11},
 'キャベツ次郎': {'kcal': 520, 'price': 110},
 'モンゴルヨーグル': {'kcal': 25, 'price': 22},
 'しんがりコーン': {'kcal': 409, 'price': 147}}

## 最適化モデルの作成
いよいよ最適化に入ります。データと予算を渡すと解いた結果を返してくれる関数にしましょう

In [23]:
def optimize(snack, budget=500):

    # モデルのインスタンス化
    model = pulp.LpProblem('knapsack', sense=LpMaximize)

    # 商品のリスト
    items = list(snack.keys())

    # 決定変数
    # お菓子iを入れるか否か
    x = {}
    for i in items:
        # x(i)は関数で中身がBinaryという意味
        x[i] = pulp.LpVariable(f'x({i})', cat='Binary')

    # 制約条件
    # 入れるお菓子の価格の合計が予算を超えない
    model += pulp.lpSum( snack[i]['price'] * x[i] for i in items ) <= budget

    # 目的関数
    # 入れるお菓子のカロリーの合計
    model += pulp.lpSum( snack[i]['kcal'] * x[i] for i in items )

    # 解く
    status = model.solve()

    return x, model.objective.value()

In [24]:
# 関数を使ってみる
x, TotalCal = optimize(snack, 500)
TotalCal

{'カンガルーのマーチ': x(カンガルーのマーチ), 'コートジボワールチョコ': x(コートジボワールチョコ), 'ガプリコ': x(ガプリコ), 'バッケス': x(バッケス), 'ハコダテポテト': x(ハコダテポテト), 'いもりこ': x(いもりこ), 'かっぱかにせん': x(かっぱかにせん), 'フェネットグミ': x(フェネットグミ), 'アルツート': x(アルツート), 'ハッピータン': x(ハッピータン), '柿の素': x(柿の素), 'たべっ子アニマル': x(たべっ子アニマル), 'あんドーナツ': x(あんドーナツ), 'かば焼き様': x(かば焼き様), 'キャベツ次郎': x(キャベツ次郎), 'モンゴルヨーグル': x(モンゴルヨーグル), 'しんがりコーン': x(しんがりコーン)}
x(しんがりコーン)
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/nozawayuta/GitHub/20211113_OptStudy/.venv/lib/python3.7/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/cf/ywg5tkw52h96hpsg57rr3bbm0000gn/T/1f6fe860b4ea43658905cfa0797f0668-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/cf/ywg5tkw52h96hpsg57rr3bbm0000gn/T/1f6fe860b4ea43658905cfa0797f0668-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 6 COLUMNS
At line 75 RHS
At line 77 BOUNDS
At line 95 ENDATA
Problem MODEL has 1 rows, 17 columns and 1

1921.0

 0.39805 (1) obj. -1812.55 iterations 2
Cbc0038I Pass  10: suminf.    0.25000 (1) obj. -1884.5 iterations 1
Cbc0038I Pass  11: suminf.    0.39805 (1) obj. -1812.55 iterations 1
Cbc0038I Pass  12: suminf.    0.39339 (2) obj. -1812.55 iterations 3
Cbc0038I Pass  13: suminf.    0.39339 (2) obj. -1812.55 iterations 0
Cbc0038I Pass  14: suminf.    0.45000 (1) obj. -1924.3 iterations 2
Cbc0038I Pass  15: suminf.    0.32006 (1) obj. -1812.55 iterations 1
Cbc0038I Pass  16: suminf.    0.32006 (1) obj. -1812.55 iterations 0
Cbc0038I Pass  17: suminf.    0.24852 (2) obj. -1812.55 iterations 2
Cbc0038I Pass  18: suminf.    0.24852 (2) obj. -1812.55 iterations 0
Cbc0038I Pass  19: suminf.    0.26667 (1) obj. -1988.4 iterations 1
Cbc0038I Pass  20: suminf.    0.37150 (1) obj. -1812.55 iterations 1
Cbc0038I Pass  21: suminf.    0.63002 (2) obj. -1812.55 iterations 6
Cbc0038I Pass  22: suminf.    0.54773 (2) obj. -1812.55 iterations 1
Cbc0038I Pass  23: suminf.    0.20000 (1) obj. -1975.8 iterations 

## 結果の確認

In [15]:
# どのお菓子を入れるのか、辞書にする
solution = {}
for i in snack.keys():
    solution[i] = x[i].value()

solution

{'カンガルーのマーチ': 1.0,
 'コートジボワールチョコ': 0.0,
 'ガプリコ': 0.0,
 'バッケス': 0.0,
 'ハコダテポテト': 0.0,
 'いもりこ': 0.0,
 'かっぱかにせん': 0.0,
 'フェネットグミ': 0.0,
 'アルツート': 0.0,
 'ハッピータン': 1.0,
 '柿の素': 0.0,
 'たべっ子アニマル': 1.0,
 'あんドーナツ': 1.0,
 'かば焼き様': 0.0,
 'キャベツ次郎': 1.0,
 'モンゴルヨーグル': 0.0,
 'しんがりコーン': 0.0}

In [25]:
# 最初のデータフレームに追加する
df['buy'] = df['name'].map(solution)
df

,name,kcal,price,buy
0,カンガルーのマーチ,336,84,1.0
1,コートジボワールチョコ,309,96,0.0
2,ガプリコ,194,102,0.0
3,バッケス,336,198,0.0
4,ハコダテポテト,446,120,0.0
5,いもりこ,299,120,0.0
6,かっぱかにせん,486,120,0.0
7,フェネットグミ,166,100,0.0
8,アルツート,344,102,0.0
9,ハッピータン,348,98,1.0


In [26]:
# コスパを出してみましょう
# 多分コスパ高いお菓子が入りやすくなっている　→　貪欲法の話
df['kcal_per_price'] = df.kcal / df.price
df.sort_values('kcal_per_price', ascending=False)

,name,kcal,price,buy,kcal_per_price
14,キャベツ次郎,520,110,1.0,4.727273
6,かっぱかにせん,486,120,0.0,4.050000
0,カンガルーのマーチ,336,84,1.0,4.000000
4,ハコダテポテト,446,120,0.0,3.716667
12,あんドーナツ,387,108,1.0,3.583333
9,ハッピータン,348,98,1.0,3.551020
8,アルツート,344,102,0.0,3.372549
11,たべっ子アニマル,330,98,1.0,3.367347
1,コートジボワールチョコ,309,96,0.0,3.218750
16,しんがりコーン,409,147,0.0,2.782313
